In [43]:
import pandas as pd
from glob import glob
import json

In [42]:
currentFiles = glob('./json/*.json')
dateList = [date[24:34] for date in currentFiles]
OWIDVaccineData = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')
joinCols = pd.read_json(currentFiles[-1])[['Location','ShortName','LongName']].drop_duplicates()

In [35]:
missingDates = []
OWIDDateList = list(OWIDVaccineData.date.unique())

for date in OWIDDateList:
    if date not in dateList:
        missingDates.append(date)
        
missingDates.sort()

In [36]:
missingDates

['2020-12-20',
 '2020-12-21',
 '2020-12-22',
 '2020-12-23',
 '2020-12-24',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30',
 '2020-12-31',
 '2021-01-01',
 '2021-01-03',
 '2021-01-09',
 '2021-01-10',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-02-15',
 '2021-02-19',
 '2021-02-20',
 '2021-02-27']

In [38]:
OWIDVaccineData.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.52,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.27,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN


In [66]:
def formatData(x):
    return {
        'Date':x.date,
        'Location': x.Location,
        'ShortName':x.ShortName,
        'LongName':x.LongName,
        'Doses_Distributed':x.total_distributed,
        'Administered_Dose1': x.people_vaccinated,
        'Administered_Dose2':x.people_fully_vaccinated
    }

In [69]:
for date in missingDates:
    dateDf = OWIDVaccineData[OWIDVaccineData.date == date]
    if (len(dateDf) == 1):
        continue
    dateDf = dateDf.merge(joinCols, how="left", left_on="location", right_on="LongName")
    formattedData = list(dateDf.apply(lambda x: formatData(x), axis=1))
    
    with open(f'./json/cdc_vaccine_data_{date}_backfillOWID.json', 'w') as outfile:
        json.dump(formattedData, outfile)